In [0]:
#fileName = dbutils.widgets.get('fileName')
fileName = 'Product.csv'
fileNameWithoutExt = fileName.split('.')[0]
print(fileNameWithoutExt)

In [0]:
import pyspark.sql.functions as F

# fill with required details of resources

sqlDbName = 'project2db'     # DB name
dbUserName = 'sqladminuser' # DB username
dbPasswordKey = 'dbpassword'  # DB password - Vault Secret's
storageAccountSASToken = 'storageAccountSASTkn' # Storageaccount SAS tocken - Vault's Secret
landingFileName = fileName
databricksScopeName = 'databricksToken01' # Azure Databrick scope credential
dbServer = 'project2dbserver'
dbServerPortNumber = '1433'
storageContainer = 'project2'
storageAccount= 'stgeaccntdlake'
landingMountPoint = '/mnt' # Mounting location


In [0]:
# Mounting the storage location
if not any(mount.mountPoint == landingMountPoint for mount in dbutils.fs.mounts()):
    dbutils.fs.mount( source = 'wasbs://{}@{}.blob.core.windows.net'.format(storageContainer, storageAccount), mount_point= landingMountPoint, extra_configs ={'fs.azure.sas.{}.{}.blob.core.windows.net'.format(storageContainer,storageAccount):dbutils.secrets.get(scope = databricksScopeName, key= storageAccountSASToken)})
    print('Successfully mounted the storage account')
else:
    print('Storage account is already mounted')

In [0]:
# Connect to Azure SQL DB
dbPassword = dbutils.secrets.get(scope = databricksScopeName, key= dbPasswordKey)
serverurl = 'jdbc:sqlserver://{}.database.windows.net:{};database={};user={};'.format(dbServer, dbServerPortNumber,sqlDbName,dbUserName)
connectionProperties = {
    'password':dbPassword,
    'driver':'com.microsoft.sqlserver.jdbc.SQLServerDriver'
}
dataframe_DB = spark.read.jdbc(url = serverurl, table = 'dbo.FileDetailsFormat', properties= connectionProperties)



In [0]:
dataframe_File = spark.read.csv('/mnt/landing/'+fileName, inferSchema=True, header=True)

# Rule 1 :- Checking for duplicate rows
errorFlag=False
errorMessage = ''
totalcount = dataframe_File.count()
print(totalcount)
distinctCount = dataframe_File.distinct().count()
print(distinctCount)
if distinctCount !=totalcount:
    errorFlag = True
    errorMessage = 'Duplication Found. Rule 1 Failed'
print(errorMessage)
    
# Rule 2 : - Checking the column format
dataframe_Filtered = dataframe_DB.filter(dataframe_DB.FileName==fileNameWithoutExt).select('ColumnName', 'ColumnDateFormat' )
rows = dataframe_Filtered.collect()
for r in rows:
    colName = r[0]
    colFormat =r[1]
    formatCount =dataframe_File.filter(F.to_date(colName, colFormat).isNotNull() ==True).count()
    if formatCount != totalcount:
        errorFlag = True
        errorMessage = errorMessage +' DateFormate is incorrect for {} '.format(colName)
    else:
        print('All rows are good for ', colName)
print(errorMessage)

if errorFlag:
    dbutils.fs.mv('/mnt/landing/'+fileName,'/mountloc/rejected/'+fileName )
    dbutils.notebook.exit('{"errorFlag": "true", "errorMessage":"'+errorMessage +'"}')
else:
    dataframe_File.write.format("delta").mode("overwrite").saveAsTable("ProductDelta")
    dbutils.fs.mv('/mnt/landing/'+fileName,'/mountloc/staging/'+fileName )
    dbutils.notebook.exit('{"errorFlag": "false", "errorMessage":"No error"}')

In [0]:
%sql
SELECT * FROM ProductDelta